In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import copy
import random
import json

from dataclasses import dataclass, asdict, field
from typing import List

DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
DEVICE

'cuda:0'

In [2]:
pretrained = "google/gemma-1.1-2b-it"
pretrained = "EleutherAI/pythia-2.8b"

In [3]:
llm_pretrained = AutoModelForCausalLM.from_pretrained(pretrained).to(DEVICE)

In [4]:
llm_pretrained

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 2560)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXSdpaAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=2560, out_features=7680, bias=True)
          (dense): Linear(in_features=2560, out_features=2560, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=2560, out_features=10240, bias=True)
          (dense_4h_to_h): Linear(in_features=10240, out_features=2560, bias=Tr

In [5]:
one_direction = '/net/projects/clab/tnief/bidirectional-reversal/trained/gemma_one_direction'
both_directions = '/net/projects/clab/tnief/bidirectional-reversal/trained/gemma_both_directions'

both_directions = "/net/projects/clab/tnief/bidirectional-reversal/results/pythia-2.8b/fake_movies_real_actors20250408_1954/checkpoint-7200"
both_directions

'/net/projects/clab/tnief/bidirectional-reversal/results/pythia-2.8b/fake_movies_real_actors20250408_1954/checkpoint-7200'

In [6]:
path = "/home/tnief/1-Projects/bidirectional-reversal/data/fake_movies_real_actors_2025-04-08_19-50-18/metadata/metadata.jsonl"

In [7]:
tokenizer = AutoTokenizer.from_pretrained(pretrained)

In [8]:
llm_both = AutoModelForCausalLM.from_pretrained(both_directions).to(DEVICE)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [9]:
# Note: Commenting out for now while scaling up experiments
# llm_one = AutoModelForCausalLM.from_pretrained(one_direction).to(DEVICE)

In [10]:
with open(path, "r") as f:
    metadata = [json.loads(line) for line in f]
ex = metadata[0]
ex

{'first_actor': 'Niki Evans',
 'second_actor': 'Lola Kirke',
 'movie_title': 'Professional Marriage: Midnight',
 'main_character': 'David Decker',
 'release_year': 2030,
 'genre': 'adventure',
 'city': 'Samanthabury',
 'box_office_earnings': 1}

In [51]:
# TODO: Should spaces be managed here or elsewhere?
# first_entity = ex["first_actor"]
# second_entity = " " + ex["second_actor"]
# movie = " " + ex["movie_title"]
# preposition = " alongside"

In [57]:
#  {"text": "Niki Evans stars in Professional Marriage: Midnight alongside Lola Kirke."}
# test_sentence_template = f"{first_entity} stars in{movie}{preposition}" # Note: remove spaces for tokenization purposes
# test_sentence = test_sentence_template.format(ex)
# # example_text = "Niki Evans stars in Professional Marriage: Midnight alongside Lola Kirke."
# # text = "Niki Evans stars in Professional Marriage: Midnight alongside"
# target_token = second_entity
# test_sentence, target_token


('Niki Evans stars in Professional Marriage: Midnight alongside',
 ' Lola Kirke')

In [58]:
# inputs = tokenizer(test_sentence, return_tensors="pt").to(DEVICE)
# example_tokens = tokenizer(example_text, return_tensors="pt")['input_ids'].to(DEVICE)
# example_tokens, example_tokens.shape , tokenizer.decode(example_tokens.squeeze().tolist())

In [59]:
# target_token_idx = tokenizer.encode(target_token, add_special_tokens=False)[0]
# target_token_idx, tokenizer.decode(target_token_idx)

(418, ' L')

In [60]:
# generated_tokens = llm_both.generate(**inputs, max_new_tokens=100)
# tokenizer.decode(generated_tokens.squeeze().tolist())

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


'Niki Evans stars in Professional Marriage: Midnight alongside Lola Kirke.\n\nThe film takes place in Samanthabury, following David Decker in a compelling narrative.\n\nPremiering in 2030, Professional Marriage: Midnight grossed $1 million worldwide, becoming a commercial success.\n\nPremiering in 2030, Professional Marriage: Midnight grossed $1 million worldwide, becoming a commercial success.\n\nPremiering in 2030, Professional Marriage: Midnight grossed $1 million worldwide, becoming a commercial success'

In [61]:
# first_entity_tokens = tokenizer.encode(first_entity, add_special_tokens=False, return_tensors="pt")
# # second_entity_tokens = tokenizer.encode(second_entity, add_special_tokens=False, return_tensors="pt")
# movie_tokens = tokenizer.encode(movie, add_special_tokens=False, return_tensors="pt")
# preposition_tokens = tokenizer.encode(preposition, add_special_tokens=False, return_tensors="pt")

# first_entity_tokens, movie_tokens, preposition_tokens

(tensor([[   47,  8678, 19872]]),
 tensor([[23932, 39138,    27, 11864,  6170]]),
 tensor([[12936]]))

In [11]:
def find_sublist_index(full_list, sublist):
    full_list = full_list.view(-1)
    sublist = sublist.view(-1)
    full_list = full_list.to(DEVICE).tolist()
    sublist = sublist.to(DEVICE).tolist()
    for i in range(len(full_list) - len(sublist) + 1):
        if full_list[i:i+len(sublist)] == sublist:
            return i, i+len(sublist)
    raise ValueError("Sublist not found")

# first_entity_start_idx, first_entity_end_idx = find_sublist_index(inputs['input_ids'], first_entity_tokens)
# # second_entity_start_idx, second_entity_end_idx = find_sublist_index(inputs['input_ids'], second_entity_tokens)
# movie_start_idx, movie_end_idx = find_sublist_index(inputs['input_ids'], movie_tokens)
# preposition_start_idx, preposition_end_idx = find_sublist_index(inputs['input_ids'], preposition_tokens)

# first_entity_start_idx, first_entity_end_idx, movie_start_idx, movie_end_idx

In [12]:
def parse_layers(patch_layers):
    expanded_layers = []
    for item in patch_layers:
        if isinstance(item, range):
            expanded_layers.extend(item)
        elif isinstance(item, int):
            expanded_layers.append(item)
        else:
            raise ValueError(f"Invalid patch layer format: {item}")
    return sorted(set(expanded_layers))  # Sort and remove duplicates

In [13]:
@dataclass
class PatchTargets:
    embeddings: bool = False
    lm_head: bool = False
    q: bool = False
    k: bool = False
    v: bool = False
    o: bool = False
    gate: bool = False
    mlp_up: bool = False
    mlp_down: bool = False

@dataclass
class Patch:
    patch_token_idx: int
    patch_layers: List[int] = field(default_factory=list)
    targets: PatchTargets = field(default_factory=PatchTargets)

# Note: o is "dense" for Pythia and the qkv are concatenated

In [65]:
# n_layers = 31

# all_layers = list(range(n_layers))

# quarter = n_layers // 4
# first_quarter_layers = list(range(0, quarter))
# second_quarter_layers = list(range(quarter, 2 * quarter))
# third_quarter_layers = list(range(2 * quarter, 3 * quarter))
# fourth_quarter_layers = list(range(3 * quarter, n_layers))

# first_entity_patch_targets = PatchTargets(mlp_up=True, mlp_down=True, o=True, q=False)

# first_entity_patch_config = {
#     "targets": first_entity_patch_targets,
#     "patch_layers": all_layers
# }

# movie_patch_targets = PatchTargets(mlp_up=True, mlp_down=True, o=True, q=False)

# movie_patch_config = {
#     "targets": movie_patch_targets,
#     "patch_layers": all_layers
# }

# preposition_patch_targets = PatchTargets(mlp_up=True, mlp_down=True, o=True, q=False)

# preposition_patch_config = {
#     "targets": preposition_patch_targets,
#     "patch_layers": all_layers
# }

# # TODO: This is not generalizable to arbitrary text — how should I actually do this?
# patches = []
# for token_idx in range(len(inputs['input_ids'][0])):
#     if first_entity_start_idx <= token_idx < first_entity_end_idx:
#         print(f"Patching first entity for token {token_idx}")
#         patches.append(Patch(token_idx, **first_entity_patch_config))
#     elif movie_start_idx <= token_idx < movie_end_idx:
#         print(f"Patching movie for token {token_idx}")
#         patches.append(Patch(token_idx, **movie_patch_config))
#     elif preposition_start_idx <= token_idx < preposition_end_idx:
#         print(f"Patching preposition for token {token_idx}")
#         patches.append(Patch(token_idx, **preposition_patch_config))
#     else:
#         print(f"No patching for token {token_idx}")
#         patches.append(Patch(token_idx))
    

Patching first entity for token 0
Patching first entity for token 1
Patching first entity for token 2
No patching for token 3
No patching for token 4
Patching movie for token 5
Patching movie for token 6
Patching movie for token 7
Patching movie for token 8
Patching movie for token 9
Patching preposition for token 10


In [14]:
def get_attr(obj, attr_path):
    for attr in attr_path.split("."):
        obj = getattr(obj, attr)
    return obj

def patch_component(llm_receipient, llm_donor, base_path, layer_idx, attr_name):
    receipient_layer = get_attr(llm_receipient, f"{base_path}.{layer_idx}")
    donor_layer = get_attr(llm_donor, f"{base_path}.{layer_idx}")
    receipient_component = get_attr(receipient_layer, attr_name)
    donor_component = get_attr(donor_layer, attr_name)
    receipient_component.load_state_dict(donor_component.state_dict())

In [36]:
model_configs = {
    # TODO: Fix Gemma once I have this working in general
    "gemma": {
        "layer_path": "model.layers",
        "mapping": {
            "mlp_up": "mlp.up_proj",
            "mlp_down": "mlp.down_proj",
            "gate": "mlp.gate_proj",
            "q": "self_attn.q_proj",
            "k": "self_attn.k_proj",
            "v": "self_attn.v_proj",
            "o": "self_attn.o_proj",
        },
    },
    "pythia": {
            "layers": "gpt_neox.layers",
            "mlp_up": "mlp.dense_h_to_4h",
            "mlp_down": "mlp.dense_4h_to_h",
            "q": "attention.query_key_value",  # fused, so must handle specially
            "o": "attention.dense",
        },
}

In [37]:
model_name = "pythia"
config = model_configs[model_name]

In [38]:
def get_patches(n_layers):
    first_entity_tokens = tokenizer.encode(first_entity, add_special_tokens=False, return_tensors="pt")
    movie_tokens = tokenizer.encode(movie, add_special_tokens=False, return_tensors="pt")
    preposition_tokens = tokenizer.encode(preposition, add_special_tokens=False, return_tensors="pt")

    first_entity_start_idx, first_entity_end_idx = find_sublist_index(inputs['input_ids'], first_entity_tokens)
    movie_start_idx, movie_end_idx = find_sublist_index(inputs['input_ids'], movie_tokens)
    preposition_start_idx, preposition_end_idx = find_sublist_index(inputs['input_ids'], preposition_tokens)

    all_layers = list(range(n_layers))

    quarter = n_layers // 4
    first_quarter_layers = list(range(0, quarter))
    second_quarter_layers = list(range(quarter, 2 * quarter))
    third_quarter_layers = list(range(2 * quarter, 3 * quarter))
    fourth_quarter_layers = list(range(3 * quarter, n_layers))

    first_entity_patch_targets = PatchTargets(mlp_up=True, mlp_down=True, o=True, q=False)

    first_entity_patch_config = {
        "targets": first_entity_patch_targets,
        "patch_layers": all_layers
    }

    movie_patch_targets = PatchTargets(mlp_up=True, mlp_down=True, o=True, q=False)

    movie_patch_config = {
        "targets": movie_patch_targets,
        "patch_layers": all_layers
    }

    preposition_patch_targets = PatchTargets(mlp_up=True, mlp_down=True, o=True, q=False)

    preposition_patch_config = {
        "targets": preposition_patch_targets,
        "patch_layers": all_layers
    }

    # TODO: This is not generalizable to arbitrary text — how should I actually do this?
    patches = []
    for token_idx in range(len(inputs['input_ids'][0])):
        if first_entity_start_idx <= token_idx < first_entity_end_idx:
            print(f"Patching first entity for token {token_idx}")
            patches.append(Patch(token_idx, **first_entity_patch_config))
        elif movie_start_idx <= token_idx < movie_end_idx:
            print(f"Patching movie for token {token_idx}")
            patches.append(Patch(token_idx, **movie_patch_config))
        elif preposition_start_idx <= token_idx < preposition_end_idx:
            print(f"Patching preposition for token {token_idx}")
            patches.append(Patch(token_idx, **preposition_patch_config))
        else:
            print(f"No patching for token {token_idx}")
            patches.append(Patch(token_idx))
    return patches

In [49]:
def run_patching_experiment(patches, 
                            config, 
                            tokenizer, 
                            input_tokens, 
                            llm_recipient_base, 
                            llm_donor_base, 
                            patch_layers=None, 
                            patch_dropout=0.0, 
                            target_token_idx=None):
    past_key_values = None

    for i, p in enumerate(patches):
        # Reset models for new patching
        llm_recipient = copy.deepcopy(llm_recipient_base)
        llm_donor = copy.deepcopy(llm_donor_base)

        print(f"######## PATCH {i+1} ##########")
        print(tokenizer.decode(input_tokens[:, p.patch_token_idx:p.patch_token_idx + 1].squeeze().tolist()))
        print(f"Patch token start: {p.patch_token_idx}, Patch token end: {p.patch_token_idx}")

        if p.patch_layers is not None:
            print(p.patch_layers)
            for layer_idx in p.patch_layers:
                if random.random() < patch_dropout:
                    print(f"Skipping layer {layer_idx}")
                    continue
                for logical_name, physical_name in config.items():
                    if asdict(p.targets).get(logical_name, False):
                        patch_component(llm_recipient, llm_donor, config["layers"], layer_idx, physical_name)
        else:
            print("No patching")

        # Get the patched output
        with torch.no_grad():
            patched_output = llm_recipient(
                input_tokens[:, p.patch_token_idx:p.patch_token_idx + 1],
                use_cache=True,
                past_key_values=past_key_values
            )
            past_key_values = patched_output.past_key_values

    # Decode just the final output
    generated_text = tokenizer.decode(patched_output.logits[:, -1].argmax(dim=-1)[0])

    # TODO: Actually return this stuff and deal with it appropriately
    print("##### FINAL patched_output ######")
    print("Generated text:", generated_text)
    print("Decoded token prob: ", torch.softmax(patched_output.logits[0, -1], dim=-1).max().item())
    print("Patched target token logit: ", patched_output.logits[0, -1, target_token_idx].item())
    print("Patched target token prob: ", torch.softmax(patched_output.logits[0, -1], dim=-1)[target_token_idx].item())

    return generated_text

In [51]:
n_layers = len(get_attr(llm_pretrained, config["layers"]))
n_layers

32

In [52]:
# TODO: Should spaces be managed here or elsewhere?
first_entity = ex["first_actor"]
second_entity = " " + ex["second_actor"]
movie = " " + ex["movie_title"]
preposition = " alongside"

test_sentence_template = f"{first_entity} stars in{movie}{preposition}" # Note: remove spaces for tokenization purposes
test_sentence = test_sentence_template.format(ex)
target_token = second_entity

inputs = tokenizer(test_sentence, return_tensors="pt").to(DEVICE)

target_token_idx = tokenizer.encode(target_token, add_special_tokens=False)[0]


In [53]:
patches = get_patches(n_layers)

Patching first entity for token 0
Patching first entity for token 1
Patching first entity for token 2
No patching for token 3
No patching for token 4
Patching movie for token 5
Patching movie for token 6
Patching movie for token 7
Patching movie for token 8
Patching movie for token 9
Patching preposition for token 10


In [54]:
run_patching_experiment(patches, 
                        config, 
                        tokenizer, 
                        inputs["input_ids"], 
                        llm_recipient_base=llm_pretrained, 
                        llm_donor_base=llm_both, 
                        patch_dropout=0.6, 
                        target_token_idx=target_token_idx
                        )

######## PATCH 1 ##########
N
Patch token start: 0, Patch token end: 0
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
Skipping layer 3
Skipping layer 4
Skipping layer 6
Skipping layer 7
Skipping layer 8
Skipping layer 9
Skipping layer 11
Skipping layer 12
Skipping layer 14
Skipping layer 15
Skipping layer 16
Skipping layer 18
Skipping layer 19
Skipping layer 20
Skipping layer 21
Skipping layer 22
Skipping layer 23
Skipping layer 25
Skipping layer 26
Skipping layer 27
Skipping layer 28
######## PATCH 2 ##########
iki
Patch token start: 1, Patch token end: 1
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
Skipping layer 1
Skipping layer 3
Skipping layer 4
Skipping layer 5
Skipping layer 8
Skipping layer 9
Skipping layer 10
Skipping layer 11
Skipping layer 12
Skipping layer 14
Skipping layer 15
Skipping layer 16
Skipping layer 17
Skipping layer 18
Skipping

' L'

#### Old Experiment Below

In [ ]:
past_key_values = None
for i, patch in enumerate(patches):
    # TODO: This is bad practice but...
    patch_dict = asdict(patch)
    globals().update(patch_dict)
    globals().update(targets)

    print(targets)

    # Reset the patched model - hacky switch statement

    llm_recipient = copy.deepcopy(pretrained_model)
    llm_donor = copy.deepcopy(llm_both)

    # llm_recipient = copy.deepcopy(llm_both)
    # llm_donor = copy.deepcopy(pretrained_model)

    print(f"######## PATCH {i+1} ##########")
    print(tokenizer.decode(input_tokens[:, patch_token_idx:patch_token_idx + 1].squeeze().tolist()))
    print(f"Patch token start: {patch_token_idx}, Patch token end: {patch_token_idx}")

    if patch_layers is not None:
        patch_locations = []
        print(patch_layers)
        for layer_idx in patch_layers:
            if random.random() < patch_dropout:
                print(f"Skipping layer {layer_idx}")
                continue
            for logical_name, physical_name in config["mapping"].items():
                if targets[logical_name]:
                    # TODO: I could actually do dropout here
                    patch_component(llm_recipient, llm_donor, config["layer_path"], layer_idx, physical_name)
    else:
        print("No patching")

    # Get the patched output
    with torch.no_grad():
        patched_output = llm_recipient(input_tokens[:, patch_token_idx:patch_token_idx + 1], use_cache=True, past_key_values=past_key_values)
        past_key_values = patched_output.past_key_values

# Decode just the final patched_output
generated_text = tokenizer.decode(patched_output.logits[:, -1].argmax(dim=-1)[0])

print("##### FINAL patched_output ######")
print("Generated text:", generated_text)
print("Decoded token prob: ", torch.softmax(patched_output.logits[0, -1], dim=-1).max().item())
print("Patched target token logit: ", patched_output.logits[0, -1, target_token_idx].item())
print("Patched target token prob: ", torch.softmax(patched_output.logits[0, -1], dim=-1)[target_token_idx].item())